# Importação das bibliotecas

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window as W

In [8]:
spark = (SparkSession.builder
        .master("local")
        .appName("PysparkSQL_SP")
        .config("spark.ui.port", "4050")
        .getOrCreate()
        )

In [9]:
spark

In [10]:
df = (spark.read.format("parquet")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("gs://projeto_final_2021/parquet_to_bq/SP/part-00000-4398f65e-4f0c-43ee-8346-9bad67a11f11-c000.snappy.parquet")
  .createOrReplaceTempView("sp")
     )

# Consultas para validação

In [5]:
#CONSULTA DA DESCRIÇÃO DA TABELA
spark.sql('DESCRIBE sp').show()

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


+--------------+---------+-------+
|      col_name|data_type|comment|
+--------------+---------+-------+
|           mes|  tinyint|   null|
|           ano| smallint|   null|
|          nome|   string|   null|
|         cargo|   string|   null|
|      situacao|   string|   null|
|salarioLiquido|    float|   null|
|        estado|   string|   null|
+--------------+---------+-------+



In [18]:
#Quantidade de servidores por cargo

spark.sql('''SELECT cargo, COUNT(nome) AS Qtd_servidores FROM sp GROUP BY cargo''').show()

+--------------------+--------------+
|               cargo|Qtd_servidores|
+--------------------+--------------+
|Diretor de Divisã...|           182|
|   TEC PROJ OBRAS VI|            43|
| ANAL DE COMUNICACAO|            24|
|DIR PART E REPR EMPR|            21|
|     Cantor de Coral|             3|
|SÇ TEC DE GASTROE...|             1|
| GER ANALISE TECNICA|             8|
|Assistente de Saú...|           104|
|           AST ADM I|           171|
|TECNICO EM SAUDE ...|           202|
|     ANL TEC INF III|            63|
| G.GESTAO DE CREDITO|            21|
|ESP DESENVOLVIMEN...|            12|
|Gerente Administr...|            11|
|AJUDANTE GERAL - ...|            35|
|DIRETOR RELAÃ‡Ã•E...|            18|
|    ASSESSOR PLENO I|            55|
|ASSISTENTE ADMINI...|            62|
|ENC.S.T.TREIN.DESENV|            20|
|Gerente Administr...|             1|
+--------------------+--------------+
only showing top 20 rows



In [6]:
#QUANTIDADE DE SERVIDORES ATIVOS POR ANO DA CAPITAL DO ESTADO DE SP
spark.sql('SELECT ano, COUNT(situacao) AS qtd_serv_ativos FROM sp WHERE situacao LIKE "ATIVO" GROUP BY ano ').show()

+----+---------------+
| ano|qtd_serv_ativos|
+----+---------------+
|2020|          26088|
|2021|           3945|
+----+---------------+



In [7]:
#QUANTIDADE TOTAL DE SERVIDORES DA CAPITAL DO ESTADO DE SP
spark.sql('SELECT ano, COUNT(situacao) AS qtd_serv_total FROM sp GROUP BY ano ').show()

+----+--------------+
| ano|qtd_serv_total|
+----+--------------+
|2020|        268203|
|2021|         25253|
+----+--------------+



In [8]:
#ANÁLISE DOS CARGOS EXISTENTES NA CAPITAL DO ESTADO DE SP
spark.sql("SELECT DISTINCT(cargo) FROM sp").show(1000,truncate=50)

+--------------------------------------------------+
|                                             cargo|
+--------------------------------------------------+
|                        Diretor de Divisão Técnica|
|                                 TEC PROJ OBRAS VI|
|                               ANAL DE COMUNICACAO|
|                              DIR PART E REPR EMPR|
|                                   Cantor de Coral|
|               SÇ TEC DE GASTROENTEROLOGIA CLINICA|
|                               GER ANALISE TECNICA|
|        Assistente de Saúde - Eletroencefalografia|
|                                         AST ADM I|
|                          TECNICO EM SAUDE NIVEL I|
|                                   ANL TEC INF III|
|                               G.GESTAO DE CREDITO|
|               ESP DESENVOLVIMENTO URBANO NÍVEL II|
|                 Gerente Administrativa Financeira|
|                 AJUDANTE GERAL - SERVIÃ‡OS GERAIS|
|                       DIRETOR RELAÃ‡Ã•ES INT

In [9]:
#ANALISAR MÉDIA SALARIAL MENSAL DE UM CARGO EM ESPECÍFICO NO ANO DE 2020
spark.sql(
"""
SELECT
    cargo,
    ROUND(AVG(salarioLiquido),2) AS media_salario,
    ano,
    mes 
FROM
    sp
WHERE
    cargo LIKE "%GUARDA%" AND ano = 2020
GROUP BY
    mes, ano, cargo
ORDER BY
    mes
"""
).show(truncate=50)

+-------------------+-------------+----+---+
|              cargo|media_salario| ano|mes|
+-------------------+-------------+----+---+
|GUARDA MUNIC.CEMIT.|      2412.26|2020|  1|
|GUARDA MUNIC.CEMIT.|      2451.57|2020|  2|
|GUARDA MUNIC.CEMIT.|      2524.52|2020|  3|
|GUARDA MUNIC.CEMIT.|       2436.3|2020|  4|
|GUARDA MUNIC.CEMIT.|      2600.33|2020|  5|
|GUARDA MUNIC.CEMIT.|      2597.62|2020|  6|
|GUARDA MUNIC.CEMIT.|      2618.08|2020|  7|
|GUARDA MUNIC.CEMIT.|      2813.96|2020|  8|
|GUARDA MUNIC.CEMIT.|      2770.89|2020|  9|
|GUARDA MUNIC.CEMIT.|      2712.09|2020| 10|
|GUARDA MUNIC.CEMIT.|      2594.92|2020| 11|
|GUARDA MUNIC.CEMIT.|       2631.0|2020| 12|
+-------------------+-------------+----+---+



In [10]:
#DADOS DO CARGO DE ENGENHEIRO
spark.sql(
"""
SELECT
    cargo,
    salarioLiquido,
    ano,
    mes 
FROM
    sp
WHERE
    cargo LIKE "ENGENHEIRO IV"
ORDER BY
    ano
"""
).show()

+-------------+--------------+----+---+
|        cargo|salarioLiquido| ano|mes|
+-------------+--------------+----+---+
|ENGENHEIRO IV|      10140.09|2020|  1|
|ENGENHEIRO IV|       9902.32|2020|  1|
|ENGENHEIRO IV|      16787.79|2020|  2|
|ENGENHEIRO IV|      9079.819|2020|  2|
|ENGENHEIRO IV|      10272.02|2020|  3|
|ENGENHEIRO IV|      10724.82|2020|  3|
|ENGENHEIRO IV|      10140.09|2020|  4|
|ENGENHEIRO IV|      9079.819|2020|  4|
|ENGENHEIRO IV|     12031.839|2020|  5|
|ENGENHEIRO IV|      9079.819|2020|  5|
|ENGENHEIRO IV|      9079.819|2020|  6|
|ENGENHEIRO IV|      10438.99|2020|  6|
|ENGENHEIRO IV|      10140.09|2020|  7|
|ENGENHEIRO IV|      9079.819|2020|  7|
|ENGENHEIRO IV|      10140.09|2020|  8|
|ENGENHEIRO IV|      9079.819|2020|  8|
|ENGENHEIRO IV|      10140.09|2020|  9|
|ENGENHEIRO IV|      9079.819|2020|  9|
|ENGENHEIRO IV|       9640.25|2020| 10|
|ENGENHEIRO IV|     8579.9795|2020| 10|
+-------------+--------------+----+---+
only showing top 20 rows



In [7]:
#CONSULTA DE UM PERÍODO EM ESPECÍFICO
spark.sql('SELECT * FROM sp WHERE ano LIKE 2021 AND mes = 1').show()

+---+----+--------------------+--------------------+--------+--------------+------+
|mes| ano|                nome|               cargo|situacao|salarioLiquido|estado|
+---+----+--------------------+--------------------+--------+--------------+------+
|  1|2021|ANTONIO FERNANDO ...|ANALISTA FISCAL D...|    null|          null|    SP|
|  1|2021|CARLA PATRICIA DO...|ANALISTA FISCAL D...|    null|          null|    SP|
|  1|2021|CARLOS ANTONIO DA...|ANALISTA FISCAL D...|    null|          null|    SP|
|  1|2021|CASSIO SANTOS PIN...|ANALISTA DE ORDEN...|    null|          null|    SP|
|  1|2021|      DANIEL DALMORO|ANALISTA DE ORDEN...|    null|          null|    SP|
|  1|2021|DIEGO MONTEIRO GO...|ANALISTA DE ORDEN...|    null|          null|    SP|
|  1|2021|  FABIANA WAKATE OTA|ANALISTA FISCAL D...|    null|          null|    SP|
|  1|2021|FABRIZIA SOARES MELO|ANALISTA FISCAL D...|    null|          null|    SP|
|  1|2021|GIZELI SILVA MARTINS|ASSISTENTE DE GES...|    null|          null|

In [19]:
# Verificar ano, cargo e salario maior que 10000 no ano de 2021
spark.sql("""
SELECT
    ano, cargo, salarioLiquido 
FROM 
    sp
WHERE
    salarioLiquido > 10000 AND ano = 2021

""").show(30)


+----+--------------------+--------------+
| ano|               cargo|salarioLiquido|
+----+--------------------+--------------+
|2021| Sup. Adm Financeiro|     17032.691|
|2021|Sup. Relações Ins...|     11612.971|
|2021|  Diretor Presidente|       15600.0|
|2021|             Diretor|      15609.82|
|2021|             Diretor|     17350.238|
|2021| Gerente de Projetos|     11795.941|
|2021| Gerente de Projetos|      20134.64|
|2021| Sup. Adm Financeiro|     11908.859|
|2021|Sup. Relações Ins...|     11612.971|
|2021| Gerente de Projetos|      12434.09|
|2021|         Assessor II|      11134.52|
|2021| Gerente de Projetos|    10361.7295|
|2021|  Diretor Presidente|      23358.06|
|2021|             Diretor|      15204.91|
|2021|             Diretor|     16075.119|
|2021|  Diretor Presidente|     15096.781|
|2021|   CHEFE DE GABINETE|       17408.0|
|2021|          PRESIDENTE|       22458.0|
|2021|   CHEFE DE GABINETE|       17408.0|
|2021|          PRESIDENTE|       22458.0|
|2021|     

In [20]:
# Verificar ano, cargo e salario maior que 10000 no ano de 2021
spark.sql("""
SELECT
    ano, cargo, salarioLiquido 
FROM 
    sp
WHERE
    salarioLiquido > 10000 AND ano = 2021

""").show(30)


+----+--------------------+--------------+
| ano|               cargo|salarioLiquido|
+----+--------------------+--------------+
|2021| Sup. Adm Financeiro|     17032.691|
|2021|Sup. Relações Ins...|     11612.971|
|2021|  Diretor Presidente|       15600.0|
|2021|             Diretor|      15609.82|
|2021|             Diretor|     17350.238|
|2021| Gerente de Projetos|     11795.941|
|2021| Gerente de Projetos|      20134.64|
|2021| Sup. Adm Financeiro|     11908.859|
|2021|Sup. Relações Ins...|     11612.971|
|2021| Gerente de Projetos|      12434.09|
|2021|         Assessor II|      11134.52|
|2021| Gerente de Projetos|    10361.7295|
|2021|  Diretor Presidente|      23358.06|
|2021|             Diretor|      15204.91|
|2021|             Diretor|     16075.119|
|2021|  Diretor Presidente|     15096.781|
|2021|   CHEFE DE GABINETE|       17408.0|
|2021|          PRESIDENTE|       22458.0|
|2021|   CHEFE DE GABINETE|       17408.0|
|2021|          PRESIDENTE|       22458.0|
|2021|     

In [23]:
# Verificar a quantidade de pagamento efetuado para cada servidor 
spark.sql('''
            SELECT 
                nome,
            COUNT
                (salarioLiquido) 
            FROM 
                sp 
            GROUP BY 
                nome 
            LIMIT 10
            ''').show()

+--------------------+---------------------+
|                nome|count(salarioLiquido)|
+--------------------+---------------------+
| RUTH EMBOAVA ARMOND|                    6|
|VINICIUS FELIPE P...|                   24|
|   ANDREIA CONSTANCI|                   24|
|       SOLANGE BIMBO|                    7|
|MARISSOL MARCELIN...|                   24|
|EDI FERREIRA DE O...|                   24|
|AMANDA FERREIRA P...|                   24|
|ANDREIA RIBEIRO D...|                   24|
|MARIA DE LOURDES ...|                   24|
|RAQUEL NASCIMENTO...|                   18|
+--------------------+---------------------+



In [26]:
#Descobrir o número total de registros de cada ano
spark.sql("select ano, count(situacao) as total_pagamentos from sp group by ano").show() 

+----+----------------+
| ano|total_pagamentos|
+----+----------------+
|2020|          384540|
|2021|          106897|
+----+----------------+



In [27]:
#Quantidade de servidores por cargo

spark.sql('''SELECT cargo, COUNT(nome) AS Qtd_servidores FROM sp GROUP BY cargo''').show()

+--------------------+--------------+
|               cargo|Qtd_servidores|
+--------------------+--------------+
|Diretor de Divisã...|           182|
|   TEC PROJ OBRAS VI|            43|
| ANAL DE COMUNICACAO|            24|
|DIR PART E REPR EMPR|            21|
|     Cantor de Coral|             3|
|SÇ TEC DE GASTROE...|             1|
| GER ANALISE TECNICA|             8|
|Assistente de Saú...|           104|
|           AST ADM I|           171|
|TECNICO EM SAUDE ...|           202|
|     ANL TEC INF III|            63|
| G.GESTAO DE CREDITO|            21|
|ESP DESENVOLVIMEN...|            12|
|Gerente Administr...|            11|
|AJUDANTE GERAL - ...|            35|
|DIRETOR RELAÃ‡Ã•E...|            18|
|    ASSESSOR PLENO I|            55|
|ASSISTENTE ADMINI...|            62|
|ENC.S.T.TREIN.DESENV|            20|
|Gerente Administr...|             1|
+--------------------+--------------+
only showing top 20 rows



In [29]:
# média salarial por estado dos servidores ativos 
spark.sql(
'''
SELECT 
    ROUND(AVG(salarioLiquido),2) AS media_salarial ,
    estado
FROM 
    sp
WHERE
    situacao = "ATIVO"
GROUP BY
    estado
'''
).show()


+--------------+------+
|media_salarial|estado|
+--------------+------+
|       5613.87|    SP|
+--------------+------+



In [30]:
#Valores gastos com o sálario ao longo dos mês ao longo do ano de 2021
spark.sql(
'''
SELECT
    mes,
    ROUND(Sum(salarioLiquido),2) AS soma_salario
FROM
    sp
WHERE 
    ano = 2021
GROUP BY 
    mes
ORDER BY
    mes ASC
 
''').show()

+---+--------------+
|mes|  soma_salario|
+---+--------------+
|  1|    8.097121E7|
|  2|   7.7065725E7|
|  3|   7.9723013E7|
|  4|   8.8025527E7|
|  5|    7.688659E7|
|  6|  1.00181983E8|
|  7|1.5032884612E8|
|  8| 8.022643507E7|
+---+--------------+



In [31]:
# Valores gastos com salário ao longo dos anos
spark.sql('''
SELECT
    ano,
    ROUND(Sum(salarioLiquido),2) as soma_salario
FROM 
    sp
GROUP BY
    ano
''').show()

+----+---------------+
| ano|   soma_salario|
+----+---------------+
|2020|1.96689897034E9|
|2021| 7.3340932919E8|
+----+---------------+



In [32]:
# Média salarial dos servidores ativos no ano de 2021
spark.sql(
"""
SELECT
    cargo,
    ROUND(AVG(salarioLiquido),2) AS media_salario,
    ano,
    mes,
    situacao
FROM
    sp
WHERE
    situacao = "ATIVO" OR situacao = "OUTRO" AND ano = 2021
GROUP BY
    mes, ano, cargo, situacao
ORDER BY
    mes
"""
).show()

+--------------------+-------------+----+---+--------+
|               cargo|media_salario| ano|mes|situacao|
+--------------------+-------------+----+---+--------+
|Ass.Gestão.Polit....|       882.15|2020|  1|   ATIVO|
|ESTAGIARIO-TEC.EVENT|      1841.09|2020|  1|   ATIVO|
|            ADVOGADO|         null|2021|  1|   ATIVO|
|           não ocupa|      3505.37|2020|  1|   ATIVO|
|MEMBRO COMITE DE ...|       3000.0|2021|  1|   OUTRO|
|ASSISTENTE TECNIC...|      3506.52|2020|  1|   ATIVO|
|CHEFE DE COORD. (...|       6342.0|2021|  1|   OUTRO|
|    GERENTE JURêDICO|      15498.0|2021|  1|   OUTRO|
|            D.SOCIAL|     15342.63|2020|  1|   ATIVO|
|Agente de Apoio -...|      2413.46|2020|  1|   ATIVO|
|  ANALISTA DESENVOLV|     10075.96|2020|  1|   ATIVO|
|  AUX PROJ OBRAS III|      6497.02|2020|  1|   ATIVO|
|  G.PLAN.CONTR. FIN.|       5484.0|2021|  1|   OUTRO|
|        ARQUITETO PL|      7490.02|2020|  1|   ATIVO|
|PRODUTOR TEC DE E...|       2939.5|2021|  1|   OUTRO|
|         